## Fabric Gateway Client
This notebook interacts with our Hyperledger Fabric blockchain, using our JavaScript module, as to simulate a Jupyter extension (i.e. the frontend).
Note that the code is executed on the backend (using a Jupyter kernel), whereas a Jupyter extension would run this code on the frontend (i.e. the web browser). This has two major consequences:
- the traffic between this client and Fabric will occur via the backend (i.e. IP routing, DNS, TLS, etc. will take place within that context);
- and the user's private key is generated and stored on the backend (rather than never leaving the web browser's memory). As long as JupyterLab is executed on the *same* system (e.g. your laptop) this does not matter (both the backend and frontend are running on the same host), but be aware when tunneling JupyterLab.

#### Admin tasks
The organization's (Fabric and IPFS) administrator has to register a user and provision the IPFS network configuration.

**User registration**  
The organization's Fabric CA admin registers user by using the command below on the Docker host (which will use our Fabric CA client application). **Note the user's (enroll) secret!**

Organization A:  
docker exec -it ca-client.fabric.localhost ./registerUser.js --profile ../connection-profiles/connection-profile-orga.yaml --org orgA --user orgAuser
Organization B:  
docker exec -it ca-client.fabric.localhost ./registerUser.js --profile ../connection-profiles/connection-profile-orgb.yaml --org orgB --user orgBuser
Organization C:  
docker exec -it ca-client.fabric.localhost ./registerUser.js --profile ../connection-profiles/connection-profile-orgc.yaml --org orgC --user orgCuser

Example of registering a user for Organization A:
 --- Fabric CA Client Application --- 
Existing wallet found at "./wallet"
Registering user "orgAuser" in organization "orgA"
An identity for the CA admin user "admin" of organization "orgA" does not exists in the wallet
Enrolling CA admin user "admin" of organization "orgA"
CA admin user password? adminpw
Successfully enrolled the CA admin user "admin" of organization "orgA"
Successfully imported identity of CA admin user "admin" of organization "orgA" into the wallet
Successfully registered user "orgAuser" in organization "orgA"
Secret of user "orgAuser": "YkrxexUdIenO"
***SECURITY NOTE***: The secret is normally not visible to the CA operator (i.e. the secret is *only* send to the user)!

**Network provisioning**  
The organization's Fabric admin uploads the IPFS private network configuration to the Fabric blockchain by using the command below on the Docker host (by using our IPFS smart contract). Note that we have to provision the addresses (in the 'multiaddr' format) of the IPFS bootstrap nodes and relays. E.g. a relay's address can be obtained via 'docker logs relay0.pnet0.orga.ipfs.localhost'.

Organization A:
docker exec cli.orga.fabric.localhost peer chaincode invoke --peerAddresses peer0.orga.fabric.localhost:7051 --peerAddresses peer0.orgc.fabric.localhost:7051 -C 'consortium-chain' -n 'consortium-cc-ipfs' -c '{"Args":["IPFSContract:createNetwork", "pnet0", "/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB", "/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f", "{\"https://cluster0.pnet0.orga.ipfs.localhost:9097\":{\"user\":\"orga\",\"password\":\"325298731aB2022aFF0964813762fC\"}}", "/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh", "{\"Users\":{},\"MSPs\":{\"MSPorgA\":\"r\"}}"]}' --tls true -o orderer0.consortium.orga.fabric.localhost:7050 --cafile /var/hyperledger/cli/crypto-orderer/tlsca.orga.fabric.localhost-cert.pem --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orga.fabric.localhost/tls/ca.crt --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orgc.fabric.localhost/tls/ca.crt

Example of Provisioning 'pnet0' for Organization A:
2022-08-14 14:36:17.590 UTC 0001 INFO [chaincodeCmd] chaincodeInvokeOrQuery -> Chaincode invoke successful. result: status:200 payload:"{\"ACL\":{\"MSPs\":{\"MSPorgA\":\"r\"},\"Users\":{}},\"BootstrapNodes\":\"/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB\",\"ClusterPinningService\":{\"https://cluster0.pnet0.orga.ipfs.localhost:9097\":{\"password\":\"325298731aB2022aFF0964813762fC\",\"user\":\"orga\"}},\"ID\":\"pnet0\",\"NetKey\":\"/key/swarm/psk/1.0.0/\\n/base16/\\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f\",\"Owner\":{\"ID\":\"orgadmin\",\"MSPId\":\"MSPorgA\"},\"Relays\":\"/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh\"}" 

Organization B:
docker exec cli.orgb.fabric.localhost peer chaincode invoke --peerAddresses peer0.orgb.fabric.localhost:7051 --peerAddresses peer0.orgc.fabric.localhost:7051 -C 'consortium-chain' -n 'consortium-cc-ipfs' -c '{"Args":["IPFSContract:createNetwork", "pnet0", "/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB", "/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f", "{\"https://cluster0.pnet0.orgb.ipfs.localhost:9097\":{\"user\":\"orgb\",\"password\":\"392449223fA2aAD134f53a8342F814\"}}", "/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh", "{\"Users\":{},\"MSPs\":{\"MSPorgB\":\"r\"}}"]}' --tls true -o orderer0.consortium.orgb.fabric.localhost:7050 --cafile /var/hyperledger/cli/crypto-orderer/tlsca.orgb.fabric.localhost-cert.pem --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orgb.fabric.localhost/tls/ca.crt --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orgc.fabric.localhost/tls/ca.crt

Organization C:
docker exec cli.orgc.fabric.localhost peer chaincode invoke --peerAddresses peer0.orga.fabric.localhost:7051 --peerAddresses peer0.orgc.fabric.localhost:7051 -C 'consortium-chain' -n 'consortium-cc-ipfs' -c '{"Args":["IPFSContract:createNetwork", "pnet0", "/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB", "/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f", "{\"https://cluster0.pnet0.orgc.ipfs.localhost:9097\":{\"user\":\"orgc\",\"password\":\"7342764f480A9feb8abd0F51463160\"}}", "/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh", "{\"Users\":{},\"MSPs\":{\"MSPorgC\":\"r\"}}"]}' --tls true -o orderer0.consortium.orgc.fabric.localhost:7050 --cafile /var/hyperledger/cli/crypto-orderer/tlsca.orgc.fabric.localhost-cert.pem --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orga.fabric.localhost/tls/ca.crt --tlsRootCertFiles /var/hyperledger/cli/crypto-peer/peer0.orgc.fabric.localhost/tls/ca.crt

### Node.js module setup
Load our Fabric client Node.js module and initialize all the variables (we set all these variables here so we can easily re-run our code/cells).

In [1]:
const fabric = require('/opt/conda/lib/node_modules/jc-fabricgw-client');

let config = null;

let CAregisterSecret = null;
let connectionDetails = null;
let response = null;

let id = null;
let network = null;
let cid = null;
let cipher = null;
let cryptKey = null;
let acl = null;

### Configuration
We use a configuration file to configure our connection to the Fabric blockchain (using the Fabric Gateway service that was added in Fabric version 2.4). Additionally, we overwrite specific settings using environment variables (e.g. the organization this notebook will connect to).

In [2]:
process.env['FABRIC_ORG'] = 'orgC';
fabric.getConfig('./fabric-client-config.yaml').then((result) => {config = result;}); // promises/async in IJavascript: https://github.com/n-riesco/ijavascript/issues/268

Configuration File: ./fabric-client-config.yaml


Promise { <pending> }

In [3]:
console.log(config);

{
  organization: 'orgC',
  mspId: 'MSPorgC',
  identity: 'orgCuser',
  idCertFile: './id/orgCuser.crt',
  idKeyFile: './id/orgCuser.key',
  caEndpoint: 'https://ca.orgc.fabric.localhost:7054',
  caTlsRootCertFile: '/mnt/crypto-config/fabric/peerOrganizations/orgc.fabric.localhost/ca/ca.orgc.fabric.localhost-cert.pem',
  caTlsVerify: true,
  caName: 'ca.orgc.fabric.localhost',
  gatewayEndpoint: 'peer0.orgc.fabric.localhost:7051',
  gatewayTlsCertFile: '/mnt/crypto-config/fabric/peerOrganizations/orgc.fabric.localhost/peers/peer0.orgc.fabric.localhost/tls/ca.crt',
  gatewayHostAlias: 'peer0.orgc.fabric.localhost',
  channel: 'consortium-chain',
  chaincode: 'consortium-cc-ipfs'
}


#### User enrollment
We have to (once) enroll our previously registered user identity (i.e. generate our public/private key pair). Note that our credentials are stored on the storage of the JupyterLab backend.

In [4]:
CAregisterSecret = 'bFEAkWJUmfKh';
fabric.execEnroll(config, CAregisterSecret);

 Enrolling... 


Promise { <pending> }

Enrollment complete!


#### Set up the connection to the Fabric gateway peer
Now that we have set up the configuration and our identity, we can connect to Fabric. Note that our configuration specifies the Fabric Gateway endpoint we want to connect to, the channel/chain/network, and the smart contract we want to use.

In [3]:
fabric.createConnection(config).then((result) => {connectionDetails = result;});

Promise { <pending> }

In [4]:
console.log(connectionDetails);

{
  gRpcClient: Client {
    [Symbol()]: [],
    [Symbol()]: [],
    [Symbol()]: undefined,
    [Symbol()]: ChannelImplementation {
      credentials: [SecureChannelCredentialsImpl],
      options: [Object],
      connectivityState: 0,
      currentPicker: [QueuePicker],
      configSelectionQueue: [],
      pickQueue: [],
      connectivityStateWatchers: [],
      configSelector: null,
      currentResolutionError: null,
      channelzEnabled: true,
      callTracker: [ChannelzCallTracker],
      childrenTracker: [ChannelzChildrenTracker],
      originalTarget: 'peer0.orgc.fabric.localhost:7051',
      callRefTimer: Timeout {
        _idleTimeout: 2147483647,
        _idlePrev: [TimersList],
        _idleNext: [TimersList],
        _idleStart: 21533,
        _onTimeout: [Function (anonymous)],
        _timerArgs: undefined,
        _repeat: 2147483647,
        _destroyed: false,
        [Symbol(refed)]: false,
        [Symbol(kHasPrimitive)]: false,
        [Symbol(asyncId)]: 34,
    

#### Configure IPFS
Get the configuration for our local IPFS node from our Fabric blockchain (i.e. the IPFS networks we have access to and their related IPFS bootstrap nodes and network key). We will use the IPFS client Python module in the [IPFS notebook](./IPFS.ipynb#Join-the-IPFS-network) to interact with IPFS.

In [19]:
fabric.listAllNetworks(connectionDetails.contract, 'MSPorgC').then((result) => {response = result;});

Promise { <pending> }

In [23]:
console.log(JSON.parse(response))
console.log(JSON.parse(response)[0].ClusterPinningService)

[
  {
    BootstrapNodes: '/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB',
    ClusterPinningService: { 'https://cluster0.pnet0.orgc.ipfs.localhost:9097': [Object] },
    ID: 'pnet0',
    NetKey: '/key/swarm/psk/1.0.0/\n' +
      '/base16/\n' +
      '63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f',
    Owner: { ID: 'orgadmin', MSPId: 'MSPorgC' },
    Relays: '/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh'
  }
]
{
  'https://cluster0.pnet0.orgc.ipfs.localhost:9097': { password: '7342764f480A9feb8abd0F51463160', user: 'orgc' }
}


### Add a data description
After having [added](./IPFS.ipynb#Add-a-file) a file via IPFS, we will upload its metadata to Fabric via our smart contract. Note that, in this example, we give read access to all users of Organization C (Fabric identities belonging to MSP 'MSPorgC').

In [18]:
id = 'testorgc.txt';
network = 'pnet0';
cid = 'QmVSVXixpttHrP7NgXw8ha9WhcbgtZNKieopZ57VeDj5yj';
cipher = 'ChaCha20';
cryptKey = '7m+Hoo77kQfF9o/u6dq4SBNKcC3ZFfXci6aZPrg+yuQ=';
acl = '{"Users":{},"MSPs":{"MSPorgC":"r","MSPorgA":"r"}}';
fabric.createData(connectionDetails.contract, id, network, cid, cipher, cryptKey, acl).then((result) => {response = result;});

Promise { <pending> }

Error: Fabric Gateway error: EndorseError: 10 ABORTED: failed to endorse transaction, see attached details for more info
    at Object.createData (/opt/conda/lib/node_modules/jc-fabricgw-client/lib/client.js:248:15)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)

In [ ]:
console.log(JSON.parse(response));

{
  ACL: { MSPs: { MSPorgA: 'r', MSPorgC: 'r' }, Users: {} },
  CID: 'QmVSVXixpttHrP7NgXw8ha9WhcbgtZNKieopZ57VeDj5yj',
  CryptCipher: 'ChaCha20',
  CryptKey: '7m+Hoo77kQfF9o/u6dq4SBNKcC3ZFfXci6aZPrg+yuQ=',
  ID: 'testorgc.txt',
  NetworkId: 'pnet0',
  Owner: { ID: 'orgCuser', MSPId: 'MSPorgC' }
}


### Retrieve a data description
We can retrieve a file's metadata from our Fabric channel/blockchain, provided we know its name and we have been granted access by the file's owner via the smart contract. Once we have obtained the metadata, we can then pass it to IPFS to [download](./IPFS.ipynb#Retrieve-a-file) the file itself.

In [24]:
id = 'orgCuser@MSPorgC/testorgc.txt';
fabric.readData(connectionDetails.contract, id).then((result) => {response = result;});

Promise { <pending> }

In [25]:
console.log(JSON.parse(response));

{
  ACL: { MSPs: { MSPorgA: 'r', MSPorgC: 'r' }, Users: {} },
  CID: 'QmVSVXixpttHrP7NgXw8ha9WhcbgtZNKieopZ57VeDj5yj',
  CryptCipher: 'ChaCha20',
  CryptKey: '7m+Hoo77kQfF9o/u6dq4SBNKcC3ZFfXci6aZPrg+yuQ=',
  ID: 'testorgc.txt',
  NetworkId: 'pnet0',
  Owner: { ID: 'orgCuser', MSPId: 'MSPorgC' }
}


**Example of trying to retrieve the data description above using a user from organization B (who is not located in the ACL):**  
Error: Fabric Gateway error: GatewayError: 2 UNKNOWN: evaluate call to endorser returned error: chaincode response 500, Access denied. User lacks permission to read data at 'orgCuser@MSPorgC/0/testorgc.txt'.

### List data descriptions
We can also list all the data descriptions, that we have access to, for the specified user:

In [29]:
id = 'orgCuser@MSPorgC';
fabric.listAllData(connectionDetails.contract, id).then((result) => {response = result;});

Promise { <pending> }

In [28]:
console.log(JSON.parse(response));

{
  ACL: { MSPs: { MSPorgA: 'r', MSPorgC: 'r' }, Users: {} },
  CID: 'QmVSVXixpttHrP7NgXw8ha9WhcbgtZNKieopZ57VeDj5yj',
  CryptCipher: 'ChaCha20',
  CryptKey: '7m+Hoo77kQfF9o/u6dq4SBNKcC3ZFfXci6aZPrg+yuQ=',
  ID: 'testorgc.txt',
  NetworkId: 'pnet0',
  Owner: { ID: 'orgCuser', MSPId: 'MSPorgC' }
}


### Delete a data description
We can delete a file's metadata from our Fabric channel/blockchain, provided we know its name and we have permission to do so (because we are the data description's owner or have write access).

In [16]:
id = 'orgCuser@MSPorgC/testorgc.txt';
fabric.deleteData(connectionDetails.contract, id).then((result) => {response = result;});

Promise { <pending> }

In [17]:
console.log(response);

{"type":"Buffer","data":[]}


 ### List the data's history
 We (the ACL entries) can view the data description's history (i.e. the transaction history of a key).

In [30]:
id = 'orgCuser@MSPorgC/testorgc.txt';
fabric.listDataHistory(connectionDetails.contract, id).then((result) => {response = result;});

Promise { <pending> }

In [31]:
console.log(JSON.parse(response));

[
  {
    TxID: 'ece0c32098a5ae0fd4284fdad2385d6df5373848d07709f6eb2965ed634cdc07',
    Timestamp: 1660507646.528,
    Data: {
      ACL: [Object],
      CID: 'QmVSVXixpttHrP7NgXw8ha9WhcbgtZNKieopZ57VeDj5yj',
      CryptCipher: 'ChaCha20',
      CryptKey: '7m+Hoo77kQfF9o/u6dq4SBNKcC3ZFfXci6aZPrg+yuQ=',
      ID: 'testorgc.txt',
      NetworkId: 'pnet0',
      Owner: [Object]
    }
  }
]


#### Close the Fabric connection

In [32]:
fabric.closeConnection(connectionDetails.gateway, connectionDetails.gRpcClient);

Promise { undefined }